In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

2023-11-27 10:47:03.290225: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return ecg_data, label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-11-27 10:47:07.850207: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78717 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:07:00.0, compute capability: 8.0


In [5]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

In [6]:
# tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [7]:
def residual_unit(x, y, n_samples_out, n_filters_out, prefix, kernel_size = 16):
    n_samples_in = y.shape[1]
    downsample = n_samples_in // n_samples_out
    n_filters_in = y.shape[2]
    
    if downsample == 1:
        y = y
    else:
        y = layers.MaxPooling1D(downsample, strides=downsample, padding='same', name = f"{prefix}_mp_opt")(y)
        
    if n_filters_in != n_filters_out:
        y = layers.Conv1D(n_filters_out, 1, padding='same', use_bias=False, name = f"{prefix}_conv_opt")(y)
        
    x = layers.Conv1D(n_filters_out, kernel_size, padding='same', use_bias=False, name = f"{prefix}_conv1")(x)
    x = layers.BatchNormalization(name = f"{prefix}_bn1")(x)
    x = layers.Activation("relu", name = f"{prefix}_act1")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout1")(x)
    x = layers.Conv1D(n_filters_out, kernel_size, strides=downsample, padding='same', use_bias=False, name = f"{prefix}_conv2")(x)

    x = layers.Add(name = f"{prefix}_add")([x,y])
    y = x
    x = layers.BatchNormalization(name = f"{prefix}_bn2")(x)
    x = layers.Activation("relu", name = f"{prefix}_act2")(x)
    x = layers.Dropout(0.2, name = f"{prefix}_dropout2")(x)
    return (x,y)
    
input_layer = tf.keras.layers.Input(shape=(5000,12), name="input")
x = input_layer
x = layers.Conv1D(64, 16, padding='same', use_bias=False, name = "conv_1")(x)
x = layers.BatchNormalization(name="bn")(x)
x = layers.Activation("relu", name="relu")(x)

x, y = residual_unit(x,x,1024,128, "res1")
x, y = residual_unit(x,y,256,196, "res2")
x, y = residual_unit(x,y,64,256, "res3")
x, _ = residual_unit(x,y,16,320, "res4")

x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(1, name="output")(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 5000, 12)]           0         []                            
                                                                                                  
 conv_1 (Conv1D)             (None, 5000, 64)             12288     ['input[0][0]']               
                                                                                                  
 bn (BatchNormalization)     (None, 5000, 64)             256       ['conv_1[0][0]']              
                                                                                                  
 relu (Activation)           (None, 5000, 64)             0         ['bn[0][0]']                  
                                                                                              

In [9]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(make_checkpoint_dir("data/models", "resnet-age"), save_best_only=True)
]

model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/50


2023-11-27 10:47:11.666549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-11-27 10:47:13.018117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-27 10:47:13.044564: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557d1b22bb00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-27 10:47:13.044607: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-11-27 10:47:13.116809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-27 10:47:13.587052: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

   7479/Unknown - 311s 40ms/step - loss: 129.7336 - mse: 129.7336 - mae: 8.9518

2023-11-27 10:52:44.311833: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8031426800233044737
2023-11-27 10:52:44.311926: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 17256161008876952570


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-11-27_10-47-08/assets


...

7479/7479 [==============================] - 324s 43ms/step - loss: 46.8220 - mse: 46.8220 - mae: 5.4319 - val_loss: 109.1105 - val_mse: 109.1105 - val_mae: 8.1799 - lr: 1.0000e-05
Epoch 38/50
7479/7479 [==============================] - 320s 43ms/step - loss: 46.7881 - mse: 46.7881 - mae: 5.4326 - val_loss: 109.2898 - val_mse: 109.2898 - val_mae: 8.1866 - lr: 1.0000e-05
Epoch 39/50
7479/7479 [==============================] - 320s 43ms/step - loss: 46.7658 - mse: 46.7658 - mae: 5.4300 - val_loss: 109.2142 - val_mse: 109.2142 - val_mae: 8.1842 - lr: 1.0000e-05
Epoch 40/50
7479/7479 [==============================] - 319s 43ms/step - loss: 46.7279 - mse: 46.7279 - mae: 5.4322 - val_loss: 109.1718 - val_mse: 109.1718 - val_mae: 8.1830 - lr: 1.0000e-05
Epoch 41/50
7479/7479 [==============================] - 319s 43ms/step - loss: 46.6019 - mse: 46.6019 - mae: 5.4201 - val_loss: 109.3761 - val_mse: 109.3761 - val_mae: 8.1897 - lr: 1.0000e-05
Epoch 42/50
7479/7479 [========================

INFO:tensorflow:Assets written to: data/models/resnet-age-2023-11-27_10-47-08/assets


7479/7479 [==============================] - 323s 43ms/step - loss: 46.0779 - mse: 46.0779 - mae: 5.3867 - val_loss: 109.0694 - val_mse: 109.0694 - val_mae: 8.1631 - lr: 1.0000e-06
Epoch 50/50
7478/7479 [============================>.] - ETA: 0s - loss: 46.1204 - mse: 46.1204 - mae: 5.3980INFO:tensorflow:Assets written to: data/models/resnet-age-2023-11-27_10-47-08/assets


INFO:tensorflow:Assets written to: data/models/resnet-age-2023-11-27_10-47-08/assets


7479/7479 [==============================] - 323s 43ms/step - loss: 46.1211 - mse: 46.1211 - mae: 5.3979 - val_loss: 108.9814 - val_mse: 108.9814 - val_mae: 8.1601 - lr: 1.0000e-06


In [10]:
model.save("data/models/resnet-age.keras")